In [26]:
from random import random, gauss, gammavariate
from math import erf, sqrt
import numpy as np
from typing import Callable 
from numpy.typing import ArrayLike

---
# Ejercicio 8
Se sortean elementos de un conjunto de datos que tiene una distribución t-student de 11 grados
de libertad. El investigador, que no conoce la forma verdadera de la distribución, asume que la misma es normal.

Analice la validez de esta suposición para muestras de tamaños 10, 20, 100 y 1000 elementos Para ello realice simulaciones numéricas e implemente el test de Kolmogorov-Smirnov a los datos simulados, asumiendo una distribución N(0,1). Presente los resultados en una tabla que contenga el número de elementos de la simulación, el valor del estadístico D y el p−valor

Ayuda: Función de probabilidad normal: Para obtener la función de probabilidad normal, se puede usar la función math.erf. Por ejemplo, la cantidad *math.erf(x/math.sqrt(2.))/2.+0.5* equivale a
$$
\int_{-\infty}^x N(0, 1)(t) dt
$$
Ayuda: Generación de números aleatorios con una distribución t-student: Utilice el siguiente código para
generar números aleatorios que siguen una distribución T-student:

In [ ]:
def cdf_Gaussian(x:float) -> float:
    """
    Función de distribución acumulada Normal
    con parámetros μ=0, σ=1

    Args:
        x (float): Valor a acumular

    Returns:
        float: Valor acumulado hasta
    """
    return erf(x / sqrt(2.0)) / 2.0 + 0.5

In [ ]:
# Generación de números aleatorios de T-student
def Tstudent(df:int) -> float:
    """
    Genera números aleatorios con distribución T-student
    con df grados de libertad.

    Args:
        df (int): Grados de libertad

    Returns:
        float: Número aleatorio
    """
    x = gauss(0.0, 1.0)
    y = 2.0 * gammavariate(0.5 * df, 2.0)
    return x / (sqrt(y / df))

In [27]:
#Definimos el estadístico
def KS_statistic(Nsamples:int, samples:ArrayLike, G:Callable[[ArrayLike], ArrayLike]) -> float:
    Fe_samples = np.arange(1, Nsamples + 1, 1) / Nsamples
    G_values = G(samples)

    Fe_minus = np.arange(0, Nsamples) / Nsamples
    Fe_plus = np.arange(1, Nsamples + 1) / Nsamples

    D_minus = G_values - Fe_minus
    D_plus = Fe_plus - G_values
    
    d = np.max(np.concatenate([D_plus, D_minus]))
    return d

#Test de Kolomogorov-Smirnov
def KS_test(Nsim:int, Nsamples:int,
            samples:ArrayLike, alpha:float,
            G:Callable[[ArrayLike], ArrayLike]):

    p_value = 0
    #Ordeño muestras 🐄
    samples = np.sort(samples)
    #Calculo estadístico inicial
    d0 = KS_statistic(Nsamples=Nsamples, samples=samples, G=G)

    for _ in range(Nsim):
        #Generamos muestras aleatorias para estimar el p_valor
        uniform_samples = np.sort([random() for _ in range(Nsamples)])
        d_sim = KS_statistic(Nsamples=Nsamples, samples=uniform_samples, G=G)
        if d_sim >= d0: 
            p_value += 1
        
    p_value = p_value / Nsim
    return (d0, p_value)

In [35]:
NSAMPLES = [10, 20, 100, 1000]
NSIM = 10_000
statistics = []
p_values = []

for Nsample in NSAMPLES:
    samples = [Tstudent(df=11) for _ in range(Nsample)]
    results = KS_test(Nsim=NSIM, Nsamples=Nsample,
                    samples=samples, alpha=0.05,
                    G=np.vectorize(cdf_Gaussian))
    statistics.append(round(results[0], 4))
    p_values.append(results[1])

In [36]:
print(f"| {'N':^5} | {'D':^9} | {'p_valor':^9} |")
print("-" * 32)

for Nsample, statistic, p_value in zip(NSAMPLES, statistics, p_values):
    print(f"| {Nsample:>5} | {statistic:^9} | {p_value:^9} |")

print("-" *32)

|   N   |     D     |  p_valor  |
--------------------------------
|    10 |  0.1774   |    1.0    |
|    20 |  0.1777   |    1.0    |
|   100 |  0.1839   |    1.0    |
|  1000 |  0.0899   |    1.0    |
--------------------------------
